In [1]:
import os
import glob
import shutil
import json
import numpy as np
from PIL import Image
from tqdm.auto import tqdm as tqdm
import numpy as np
import pandas as pd
from itertools import product
import re
np.random.seed(42)

In [2]:
method = 'GAN/20231105_gan_1x'
root_dir = os.path.join('..','data', 'data_aug')


data_dir = os.path.join(root_dir, method)
annot_path = os.path.join(root_dir, method, 'labels')

out_dir = data_dir
images_path = os.path.join(root_dir, method, 'images')

json_name = 'coco_format_%s_%s.json'

In [3]:
images = glob.glob(images_path+'/**/*.png', recursive=True)
images

['../data/data_aug/GAN/20231105_gan_1x/images/video04_frame_0600.png',
 '../data/data_aug/GAN/20231105_gan_1x/images/video04_frame_0624.png',
 '../data/data_aug/GAN/20231105_gan_1x/images/video04_frame_0648.png',
 '../data/data_aug/GAN/20231105_gan_1x/images/video04_frame_0672.png',
 '../data/data_aug/GAN/20231105_gan_1x/images/video04_frame_0696.png',
 '../data/data_aug/GAN/20231105_gan_1x/images/video04_frame_0744.png',
 '../data/data_aug/GAN/20231105_gan_1x/images/video04_frame_0768.png',
 '../data/data_aug/GAN/20231105_gan_1x/images/video04_frame_0912.png',
 '../data/data_aug/GAN/20231105_gan_1x/images/video04_frame_1488.png',
 '../data/data_aug/GAN/20231105_gan_1x/images/video04_frame_1512.png',
 '../data/data_aug/GAN/20231105_gan_1x/images/video04_frame_1608.png',
 '../data/data_aug/GAN/20231105_gan_1x/images/video04_frame_1632.png',
 '../data/data_aug/GAN/20231105_gan_1x/images/video06_frame_0000.png',
 '../data/data_aug/GAN/20231105_gan_1x/images/video04_frame_0432.png',
 '../d

In [4]:
root = images[0].split(method)[0]
root

'../data/data_aug/'

In [5]:
root = os.path.join(root, method)
root

'../data/data_aug/GAN/20231105_gan_1x'

In [6]:
images[0][-14:]

'frame_0600.png'

In [9]:
# separating images by video folders
r = re.compile(r'video\d{2}')

for image_path in tqdm(images):
    root = image_path.split(method)[0]
    root = os.path.join(root, method)
        
    video_name = r.findall(image_path)[0]
    frame_name = image_path[-14:]
    
    os.makedirs(os.path.join(images_path, video_name),  exist_ok=True)    
    shutil.move(image_path, os.path.join(images_path, video_name, frame_name))    
    
    
    os.makedirs(os.path.join(images_path.replace('images', 'labels'), video_name),  exist_ok=True) 
    label_path = image_path.replace('images', 'labels').replace('.png', '.txt')    
    shutil.move(label_path, os.path.join(images_path.replace('images', 'labels'), video_name, frame_name.replace('.png', '.txt')))    
    

  0%|          | 0/171 [00:00<?, ?it/s]

In [10]:
def get_img_size(datapath):
    img_pil = Image.open(datapath)
    width, height = img_pil.size
    return width, height


def get_groundtruth(img_path, annotation_folder):
    annot_path = None
    if annotation_folder is not None:
        # look for annotation file
        annot_path = _find_annot_file(img_path, annotation_folder)

    if annot_path is not None:
        frame_number = _get_frame_number(img_path)
        annotation = AnnotationImage(frame_number, annot_path)
        boxes, labels = annotation.get_bboxes_labels()

        return boxes, labels

    return [], []

In [11]:
k = 0 #fold
file_set = '../data/v1/train_sets_kfold_v1.0.xls'
df_sets = pd.read_excel(file_set, sheet_name=f'train_sets_k{k}')
df_sets

,Video,seq,bottle,bucket,pool,puddle,tire,water_tank,local,altitude,train,val,test
0,20180905_rectified_DJI_0010,video01,NaN,NaN,NaN,NaN,NaN,NaN,Campo Gremio,10,NaN,NaN,1.0
1,20180919_rectified_DJI_0019,video02,NaN,NaN,NaN,NaN,NaN,NaN,Atras Bloco I,15,NaN,NaN,1.0
2,20180919_rectified_DJI_0021,video03,NaN,NaN,NaN,NaN,NaN,NaN,Acima bloco I (video longo),40,1.0,NaN,NaN
3,20181022_rectified_DJI_0025,video04,NaN,NaN,NaN,NaN,NaN,NaN,aeroclube,15,NaN,1.0,NaN
4,20181022_rectified_DJI_0031,video05,NaN,NaN,NaN,NaN,NaN,NaN,Atras FAU (wild),10,NaN,NaN,1.0
5,20181029_rectified_DJI_0034,video06,NaN,NaN,NaN,NaN,NaN,NaN,CCMN,15,1.0,NaN,NaN
6,20181029_rectified_DJI_0038,video07,NaN,NaN,NaN,NaN,NaN,NaN,frente FAU,20,1.0,NaN,NaN
7,20181112_rectified_DJI_0041,video08,NaN,NaN,NaN,NaN,NaN,NaN,Terreno Atras Bloco I,10,NaN,1.0,NaN
8,20181112_rectified_DJI_0043,video09,NaN,NaN,NaN,NaN,NaN,NaN,Acima bloco I,40,NaN,NaN,1.0
9,20190601_rectified_DJI_0003,video10,NaN,NaN,NaN,NaN,NaN,NaN,Tubiacanga,40,1.0,NaN,NaN


In [12]:
 for (dirpath, dirnames, filenames) in tqdm(os.walk(os.path.join(data_dir))):
        print(dirpath)
        print(dirnames)
        print(filenames)
        
        print('***')

0it [00:00, ?it/s]

../data/data_aug/GAN/20231105_gan_1x
['images', 'labels']
[]
***
../data/data_aug/GAN/20231105_gan_1x/images
['video06', 'video07', 'video08', 'video10', 'video11', 'video12', 'video04']
[]
***
../data/data_aug/GAN/20231105_gan_1x/images/video06
[]
['frame_0000.png', 'frame_0024.png', 'frame_0048.png', 'frame_0072.png', 'frame_1032.png', 'frame_1056.png', 'frame_1080.png', 'frame_1416.png', 'frame_1440.png', 'frame_1464.png', 'frame_1488.png', 'frame_1512.png', 'frame_1536.png', 'frame_1560.png', 'frame_1584.png']
***
../data/data_aug/GAN/20231105_gan_1x/images/video07
[]
['frame_0336.png', 'frame_0360.png', 'frame_0384.png', 'frame_1800.png', 'frame_1824.png', 'frame_1848.png', 'frame_2496.png', 'frame_2520.png', 'frame_2544.png', 'frame_2568.png', 'frame_3936.png', 'frame_3960.png', 'frame_3984.png', 'frame_4008.png', 'frame_4032.png', 'frame_4056.png', 'frame_4080.png']
***
../data/data_aug/GAN/20231105_gan_1x/images/video08
[]
['frame_0144.png', 'frame_0168.png', 'frame_0192.png', 

In [21]:
method

'GAN/20231105_gan_1x'

In [22]:
import re

include_reals = True # please, manually check if it is already included in the file

mask = df_sets['test'] == True
videos_train_val = df_sets[~mask]['seq'].tolist()

sets = [f'train{n}' for n in np.arange(len(videos_train_val))]
sets += ['train+val']


category_dict = {
    #'__background__': 0,
    'tire': 0,
    }

for subset in tqdm(sets):
    print(f'Starting {method, subset}')
    
    if 'train+val' in subset:
        videos_set = videos_train_val
        
        
    elif 'train' in subset:
        idx_val = int(subset[-1])
        videos_set = [x for i, x in enumerate(videos_train_val) if i != idx_val]        
        
    ann_dict = {}
    images = []
    annotations = []
        
    img_id = 0
    ann_id = 0
    cat_id = 1
    
#     filenames = glob.glob(os.path.join(data_dir, data_set,'*.png'))
    
    for (dirpath, dirnames, filenames) in tqdm(os.walk(os.path.join(data_dir))):
        
        if len(filenames) == 0:
            continue
    
        video_name = dirpath.split('/')[-1]                
        if video_name not in videos_set:            
            continue
        
        print(video_name)
        for filename in tqdm(filenames):               
            if filename.lower().endswith('.png'):                          
                image = {}
                
                img_name_short = os.path.join('images', video_name, filename)
                img_name = os.path.join(dirpath, filename)
                
                    
                width, height = get_img_size(img_name)
                
                image['file_name'] = img_name_short
                image['width'] = width
                image['height'] = height
                image['id'] = img_id
                                
                #read .txt in images folder
                if include_reals:                    
                    anno_filepath = os.path.join(root_dir, "GAN", "labels_reals", 
                                                 f"{video_name}_{filename.replace('.png','.txt')}")
                    
                    if os.path.isfile(anno_filepath):
                        num_real_in_img = 0
                        
                        with open(anno_filepath, 'r') as marks:
                            for mark in marks.readlines():  
                                rect = mark.replace('\n','').split('\t') 
                                
                                
                                
                                if len(rect) <=1:                                    
                                    continue
                                    
                                objid,rcx,rcy,rw,rh = map(float,rect)
                                x0 = int((rcx-rw/2)*width) #hardcoded pq veio assim da annot
                                y0 = int((rcy-rh/2)*height)
                                w = int(rw*width)
                                h = int(rh*height)

                                ann = {}
                                ann['id'] = ann_id
                                ann['image_id'] = img_id
                                ann['category_id'] = 0
                                ann['iscrowd'] = 0
                                ann['area'] = w*h
                                ann['bbox'] = (x0,y0,w,h)

                                annotations.append(ann)
                                ann_id += 1    
                                num_real_in_img += 1      
                                
                #read .txt in labels folder                                
                anno_filepath = os.path.join(data_dir, 'labels', video_name, filename.replace('.png','.txt'))                

                with open(anno_filepath, 'r') as marks:

                    for mark in marks.readlines(): 
                        print(anno_filepath)
                        print(mark)
                        
                        rect = mark.replace('\n','')
#                       
                        if '\t' in rect:
                            rect = rect.split('\t')                                                                  
                        if ' ' in rect:
                            rect = rect.split(' ')  
                        
                        print(rect)
                        print('\n')

                        if len(rect) <=1:            
                            continue

                        objid,rcx,rcy,rw,rh = map(float,rect)
                    
                        
                        x0 = int((rcx-rw/2)*width) 
                        y0 = int((rcy-rh/2)*height)
                        w = int(rw*width)
                        h = int(rh*height)                            

                        ann = {}
                        ann['id'] = ann_id
                        ann['image_id'] = img_id
                        ann['category_id'] = 0
                        ann['iscrowd'] = 0
                        ann['area'] = w*h
                        ann['bbox'] = (x0,y0,w,h)

                        annotations.append(ann)
                        ann_id += 1
                        
                images.append(image)
                img_id += 1

            
    ann_dict['images'] = images
    categories = [{"id": category_dict[name], "name": name} for name in category_dict]
    ann_dict['categories'] = categories
    ann_dict['annotations'] = annotations
    print("Num categories: %s" % len(categories))
    print("Num images: %s" % len(images))
    print("Num annotations: %s" % len(annotations))
    
    
    with open(os.path.join(out_dir, json_name % ('GAN', subset)), 'w') as outfile:
        outfile.write(json.dumps(ann_dict))

  0%|          | 0/9 [00:00<?, ?it/s]

Starting ('GAN/20231105_gan_1x', 'train0')


0it [00:00, ?it/s]

video06


  0%|          | 0/15 [00:00<?, ?it/s]

...

video12


  0%|          | 0/25 [00:00<?, ?it/s]

video04


  0%|          | 0/14 [00:00<?, ?it/s]

Num categories: 1
Num images: 171
Num annotations: 812


In [23]:
out_dir

'../data/data_aug/GAN/20231105_gan_1x'